# Simple link prediction
In this notebook, we'll implement a simple RF-based link prediction method where we'll train on a previous year to predict the subsequent year. In past iterations, we have trained on all years prior to 2020 and then predicted on all years past 2020, but this appeared to cause data leakage in a way we didn't understand. So for the sake of illustration, here I'll just take the year with the most data and predict the subsequent year, to see if this clears upt he data leakage issue.

In [1]:
import networkx as nx
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from statistics import median
from itertools import combinations
from tqdm import tqdm
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

## Read in Data

In [2]:
graph = nx.read_graphml('../data/kg/all_drought_dt_co_occurrence_graph_02May2024.graphml')

## Formulation of ML problem
We want to predict desiccation edges that appear in the future of the graph. In order to prevent data leakage, we'll formulate this problem as predicting just edges that appear in the next year. We can then string together sets of predictions to perform forecasting farther into the future.

Starting in 2015, we'll slice the graph for each year. The training set features will be built from the year with the most new links, and the training labels will be taken from the subsequent year. The test set features will be built from the same training features year, but the links taken from the year following the training links.

## Feature extraction
In a past notebook (now removed from this repo, but still in thie history) we explored maunal feature extraction, so I will just jump straight to extracting them with the code from that notebook. We do, however, need to adjust this code to only use data up to one year for feature extraction for both training and testing.

In [9]:
def get_paired_features(pair_names, edges, per_node_features):
    """
    Get node pair features.
    """
    all_paired_features = {}
    for n1, n2 in pair_names:
        pair_features = {}
        # Check if it's an edge
        if ((n1, n2) in edges.keys()) or ((n2, n1) in edges.keys()):
            is_edge = True
            try:
                edge_attrs = edges[(n1, n2)]
                key_order = (n1, n2)
            except KeyError:
                edge_attrs = edges[(n2, n1)]
                key_order = (n2, n1)
        else:
            is_edge = False
            key_order = (n1, n2)
        # Merge the attributes for the individual nodes
        for k, v in per_node_features[n1].items():
            pair_features[f'{k}_n1'] = v
        for k, v in per_node_features[n2].items():
            pair_features[f'{k}_n2'] = v
        # Add composite features/labels
        if is_edge:
            pair_features['are_connected'] = True
            if edge_attrs['is_drought']:
                pair_features['is_drought_edge'] = True
                if edge_attrs['is_desiccation']:
                    pair_features['is_desiccation_edge'] = True
                else:
                    pair_features['is_desiccation_edge'] = False
            else:
                pair_features['is_drought_edge'] = False
                if edge_attrs['is_desiccation']:
                    pair_features['is_desiccation_edge'] = True
                else:
                    pair_features['is_desiccation_edge'] = False
            pair_features['year_first_connected'] = edge_attrs['first_year_mentioned']
        else:
            pair_features['are_connected'] = False
            pair_features['is_drought_edge'] = False
            pair_features['is_desiccation_edge'] = False
        all_paired_features[key_order] = pair_features
        
    return all_paired_features

In [10]:
def get_node_features(graph, cut_years):
    """
    Get the per-node features for each year's instances.
    """
    year_per_node_features = {}
    year_graphs = {}
    for cut_year in cut_years:
        # Get rid of anything after the cut year in the graph
        pre_cutoff_graph = nx.Graph()
        new_nodes = [(n, attrs) for n, attrs in graph.nodes(data=True)
                               if int(attrs['first_year_mentioned']) < cut_year]
        new_edges = [(e1, e2, attrs) for e1, e2, attrs in graph.edges(data=True)
                               if int(attrs['first_year_mentioned']) < cut_year]
        _ = pre_cutoff_graph.add_nodes_from(new_nodes)
        _ = pre_cutoff_graph.add_edges_from(new_edges)
        year_graphs[cut_year] = pre_cutoff_graph
    
        # Get the features for each node
        per_node_features = defaultdict(dict)

        # Degree
        g_degree = pre_cutoff_graph.degree()
        for n, deg in g_degree:
            per_node_features[n]['degree'] = deg

        # Entity type, year, number of node mentions
        for n, attrs in pre_cutoff_graph.nodes(data=True):
            per_node_features[n]['ent_type'] = attrs['ent_type']
            per_node_features[n]['node_year_first_mentioned'] = attrs['first_year_mentioned']

        # Edge-based feature basics
        years_added = defaultdict(list)
        node_des_only_edge_counts = defaultdict(int)
        for e1, e2, attrs in tqdm(pre_cutoff_graph.edges(data=True)):
            # Years
            years_added[e1].append(int(attrs['first_year_mentioned']))
            years_added[e2].append(int(attrs['first_year_mentioned']))
            # Counts for edge proportion
            if attrs['is_desiccation']:
                if not attrs['is_drought']:
                    node_des_only_edge_counts[e1] += 1
                    node_des_only_edge_counts[e2] += 1
        
        # Process edge based features
        for n in pre_cutoff_graph.nodes:
            # Years
            try:
                years = years_added[n]
                counted = Counter(years)
                per_node_features[n]['avg_year_additions'] = sum(counted.values())/len(counted)
                per_node_features[n]['median_year_added'] = median(years)
            except ZeroDivisionError:
                per_node_features[n]['avg_year_additions'] = 0
                per_node_features[n]['median_year_added'] = 0
            # Edge proportion
            try:
                des_count = node_des_only_edge_counts[n]
                per_node_features[n]['des_only_prop'] = des_count/g_degree[n]
            except ZeroDivisionError:
                per_node_features[n]['des_only_prop'] = 0
        year_per_node_features[cut_year] = per_node_features
        
    return year_per_node_features, year_graphs

In [11]:
def get_positive_and_negative_pairs(graph, cut_years):
    """
    Get the pair names for positive and negative instances. Negative instances are negative over all time (for which
    the graph contains data), but are balanced for each year and only contain feature information for the same time
    period as the positive instances for that year.
    """
    positives = {}
    negatives = {}
    all_negative_names = []
    node_years = nx.get_node_attributes(graph, 'first_year_mentioned')
    for cut_year in cut_years:
        
        # Get positive instances, which are edges added in the cut year
        positive_pair_names = [(e1, e2) for e1, e2, attrs in graph.edges(data=True)
                               if attrs['is_desiccation'] and (int(attrs['first_year_mentioned']) == cut_year)]
        positive_pair_names = [pair for pair in positive_pair_names if (int(node_years[pair[0]]) < cut_year)
                              and (int(node_years[pair[1]]) < cut_year)]
        print(f'There are {len(positive_pair_names)} positive pairs for year {cut_year}.')
        
        # Get negative instances, which are pairs of nodes that never get an edge
        negative_pair_candidates = combinations(list(graph.nodes), 2)
        negative_pair_names = []
        for pair in negative_pair_candidates:
            if len(negative_pair_names) == len(positive_pair_names):
                break
            else:
                if not graph.has_edge(pair[0], pair[1]): # If they're never connected
                    if pair not in all_negative_names: # And it's not already a negative
                        if (int(node_years[pair[0]]) < cut_year) and (int(node_years[pair[1]]) < cut_year): # And the nodes exist prior to this year
                            negative_pair_names.append(pair) # Add it as a negative
                            all_negative_names.append(pair)
        print(f'A corresponding {len(negative_pair_names)} negative instances have been constructed for year {cut_year}.')
    
        # Add to the dictionaries for the year
        positives[cut_year] = positive_pair_names
        negatives[cut_year] = negative_pair_names
        
    return positives, negatives

In [12]:
def build_feature_table(graph, cut_years=[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023], test_cutoff=2020):
    """
    Build the feature table. Current features are:
        Node degree
        Entity type
        Proportion of edges for each node that are desiccation or drought
        Rate of edge addition for each node (using first year of mention)
        Whether or not the other two nodes are already connected by a drought edge
    """
    # Get the pair names for positive and negative instances from full graph, using cutoff
    print('\nGetting positive and negative instances...')
    positives, negatives = get_positive_and_negative_pairs(graph, cut_years)
    
    # Build single-node feature vectors
    print('\nWrangling features for each node...')
    year_per_node_features, year_graphs = get_node_features(graph, cut_years)

    # Build paird feature vectors
    year_paired_feature_dfs = {}
    for cut_year in cut_years:
        
        # Get edges from this year
        edges = {e[0]: e[1] for e in year_graphs[cut_year].edges(data=True)}
        
        # Get positive paired feature vectors
        pos_instances = get_paired_features(positives[cut_year], edges, year_per_node_features[cut_year])
        pos_df = pd.DataFrame.from_records(pos_instances).T
        pos_df['label'] = 1
        pos_df['year'] = cut_year
    
        # get negative positive paired feature vectors
        neg_instances = get_paired_features(negatives[cut_year], edges, year_per_node_features[cut_year])
        neg_df = pd.DataFrame.from_records(neg_instances).T
        neg_df['label'] = 0
        neg_df['year'] = cut_year
        
        year_paired_feature_dfs[cut_year] = [pos_df, neg_df]

    # Make train and test dataframes
    print('\nMaking feature table...')
    year_paired_feature_dfs = {cut_year: pd.concat([dfs[0], dfs[1]]) for cut_year, dfs in year_paired_feature_dfs.items()}
    train_df = pd.concat([df for cut_year, df in year_paired_feature_dfs.items() if cut_year < test_cutoff])
    test_df = pd.concat([df for cut_year, df in year_paired_feature_dfs.items() if cut_year >= test_cutoff])
    print(f'The training dataframe has shape {train_df.shape}.')
    print(f'The test dataframe has shape {test_df.shape}.')
    
    print('\nDone!')
    return train_df, test_df

NameError: name 'unit_test_graph' is not defined